In [20]:
from datascience import *
import numpy as np
import pandas as pd
import json

In [21]:
def truncate_names(name):
    truncated = name.split(" ")
    modified = 0
    
    # abbrev
    if np.all([True if x.isupper() else False for x in truncated[0]]):
        if truncated[1][0] == truncated[0][0]:
            truncated = truncated[1:]
            modified = 1
        else:
            modified = 0
    has_p = [True if x.find("(") != -1 else False for x in truncated]
    if np.any(has_p):
        index = ([i for i, x in enumerate(has_p) if x][0])
        truncated = " ".join(truncated[0:index])
        modified = 1
    
    if not modified:
        return name
    return " ".join(truncated) if type(truncated) == list else truncated

def nodes_links(table, mapping):
    # NODES
    industry_json = {}
    nodes = []
    links = []
    nodes_details = {}
    sponsors = table.column('Sponsor')
    depts = table.column('Department')
    total = table.column('Total')
    index = 0
    for i in range(table.num_rows):
        node_json = {}
        if sponsors[i] not in nodes_details.keys():
            nodes_details[sponsors[i]] = index
            node_json['name'] = sponsors[i]
            vals = [(val['abbrev'], val['cat']) for val in mappings['nodes'] if node_json['name'] == val['name']]
            node_json['type'] = "sponsors"
            if len(vals):
                node_json['cat'] = vals[0][1]
                node_json['abbrev'] = vals[0][0]
            else:
                node_json['cat'] = ""
                node_json['abbrev'] = ""
            index += 1
            nodes = nodes + [node_json]
        node_json1 = {}
        if depts[i] not in nodes_details.keys():
            nodes_details[depts[i]] = index
            node_json1['name'] = depts[i]
            vals = [(val['abbrev'], val['cat']) for val in mappings['nodes'] if node_json1['name'] == val['name']]
            node_json1['type'] = "targets"
            if len(vals):
                node_json1['cat'] = vals[0][1]
                node_json1['abbrev'] = vals[0][0]
            else:
                node_json1['cat'] = ""
                node_json1['abbrev'] = ""
            index += 1
            nodes = nodes + [node_json1]
    # LINKS
    for i in range(table.num_rows):
        link_json = {}
        link_json['source'] = nodes_details[sponsors[i]]
        link_json['target'] = nodes_details[depts[i]]
        link_json['value'] = total[i]
        links += [link_json]
    industry_data = {}
    industry_data['nodes'] = nodes
    industry_data['links'] = links
    return industry_data

In [42]:
combined_phoebe_conc_pd_ds.where('Sponsor', 'BP Group (BP America, BP Exploration, British Petroleum)')

Unnamed: 0,Sponsor,Control Unit,Division,Department,Sponsor Class,2014,2015,2016
206,"BP Group (BP America, BP Exploration, British Petroleum)",Colleges,College of Chemistry,Chemical and Biomolecular Engineering,Industry,0,1350000,0
207,"BP Group (BP America, BP Exploration, British Petroleum)",Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,-200000,0,0
208,"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,Industry,0,0,5000000
209,"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,10000,0,0
210,"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,Vice Chancellor for Research ORUs,Null,Industry,0,0,0


In [43]:
combined_phoebe_conc_pd_ds = Table().read_table('data/research_funding_clean.csv')
combined_phoebe_conc_pd_ds.show(5)

Unnamed: 0,Sponsor,Control Unit,Division,Department,class_clean,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,22408
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,5000
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,0,0,0


In [44]:
combined_phoebe_conc_pd_ds.relabel('class_clean', 'Sponsor Class')

Unnamed: 0,Sponsor,Control Unit,Division,Department,Sponsor Class,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,22408
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,5000
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,0,0,0
5,AYUSA International,Vice Chancellor - Research,Academic Research Units,IURD Institute of Urban & Regional Development,Not for Profit,54481,60000,0
6,Aalborg University Copenhagen,Vice Chancellor - Research,Vice Chancellor for Research ORUs,IBER Institute of Business & Economic Research,Not for Profit,0,53015,0
7,Abraham J. & Phyllis Katz Foundation,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Not for Profit,0,0,0
8,"Abt Associates, Inc.",Colleges,CNR College of Natural Resources,Null,Industry,73942,0,0
9,Academy of Applied Science,Colleges,CNR College of Natural Resources,"Environmental Science, Policy & Management",Not for Profit,0,4000,4000


In [52]:
industry = combined_phoebe_conc_pd_ds.where('Sponsor Class', "Industry")
industry = industry.with_column("Total", industry.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))\
                    .select([1, 2, 3, 4, 'Total'])\
                    .sort('Total', descending=True)
#                     .where('Total', are.above(0))
industry.show(5)

# 

Sponsor,Control Unit,Division,Department,Total
BASF Corporation,Colleges,College of Chemistry,Chemistry Department,9069312
Aduro Biotech,L&S College of Letters & Sciences,L&S Biological Sciences,Cancer Research Laboratory,7500000
"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,5000000
Intel Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),2904754


In [53]:
industry_nonnull = industry.where(industry.column("Department") != "Null")\
                           .select(['Sponsor','Department', 'Total'])
industry_null = industry.where('Department', 'Null')\
                        .select(["Sponsor", 'Division', "Total"])\
                        .relabel("Division", "Department")
industry_all = industry_nonnull.append(industry_null)
industry_all = industry_all.with_columns(['Sponsor', 
                                        industry_all.apply(truncate_names, ['Sponsor']),
                                         'Department',
                                        industry_all.apply(truncate_names, ['Department'])])
industry_all.show(3)

Sponsor,Department,Total
BASF Corporation,Chemistry Department,9069312
Aduro Biotech,Cancer Research Laboratory,7500000
BP Group,Energy Biosciences Institute,5000000


In [54]:
top50_sponsors = industry_all.group('Sponsor', sum)\
                            .sort('Total sum', descending=True)\
                            .take(np.arange(50))\
                            .column('Sponsor')

In [55]:
top50_sponsors

array(['BASF Corporation', 'Aduro Biotech', 'Intel Corporation',
       'BP Group', 'Pfizer, Inc.', 'Chevron Corporation',
       'Samsung Electronics Company, Ltd.', 'Toyota Motor Corporation',
       'Dow Chemical Company', 'AstraZeneca Inc.', 'Qualcomm, Inc.',
       'GlaxoSmithKline', 'Wellspring Advisors LLC',
       'Hoffmann-La Roche LTD.', 'Altria Group, Inc.', 'BAE Systems',
       'Oak Ridge National Laboratory', 'TATA Consultancy Services',
       'Volkswagen Group of America, Inc.',
       'Robert Bosch Corporation GMBH', 'Ford Motor Company',
       'Panasonic Corporation', 'Charles River Analytics',
       'Taiwan Semiconductor Manufacturing Company. Ltd',
       'State Street Corporation', 'Siemens Corporation',
       'InvenSense, Inc.', 'Korea Electric Power Corporation',
       'Google, Inc.', 'Sandia National Laboratory',
       'International Business & Technical Consultants, Inc.',
       'Denso Corporation', 'Exxon Mobil Corporation',
       'NVIDIA Corporation', 

In [56]:
industry_all = industry_all.with_column('Sponsor Copy', industry_all.column('Sponsor'))
industry_all = industry_all.with_column('Sponsor',
                           industry_all.apply(lambda x: x if x in top50_sponsors else 'Other', ['Sponsor']))
industry_all

Sponsor,Department,Total,Sponsor Copy
BASF Corporation,Chemistry Department,9069312,BASF Corporation
Aduro Biotech,Cancer Research Laboratory,7500000,Aduro Biotech
BP Group,Energy Biosciences Institute,5000000,BP Group
Intel Corporation,Electrical Engineering & Computer Sciences,3929000,Intel Corporation
"Pfizer, Inc.",The California Institute for Quantitative Biosciences,2904754,"Pfizer, Inc."
Chevron Corporation,Chemical and Biomolecular Engineering,2825291,Chevron Corporation
Intel Corporation,Engineering Research Support Organization,2409614,Intel Corporation
AstraZeneca Inc.,Innovative Genomics Initiative,2340000,AstraZeneca Inc.
Wellspring Advisors LLC,Center of Evaluation for Global Action,1879998,Wellspring Advisors LLC
GlaxoSmithKline,Chemical and Biomolecular Engineering,1810954,GlaxoSmithKline


In [58]:
industry_all.group('Sponsor', sum).sort('Total sum', descending=True).show()

Sponsor,Department sum,Total sum,Sponsor Copy sum
Other,,26794649,
BASF Corporation,,10739602,
Aduro Biotech,,7926154,
Intel Corporation,,6338614,
BP Group,,6160000,
"Pfizer, Inc.",,4597764,
Chevron Corporation,,4486291,
"Samsung Electronics Company, Ltd.",,3984982,
Toyota Motor Corporation,,2898560,
Dow Chemical Company,,2715124,


In [28]:
# Making one aggregate Other link for each Department
industry_grouped_other = industry_all.where('Sponsor', 'Other')\
                                     .group('Department', sum)\
                                     .relabel('Total sum', 'Total')\
                                     .select(['Department', 'Total'])
industry_grouped_other = industry_grouped_other.with_column('Sponsor', 
                                                    ['Other']*industry_grouped_other.num_rows)

# Combining top50 with the others 
industry_grouped_all = industry_all.where(industry_all.column('Sponsor') != "Other")\
                                   .drop('Sponsor Copy')\
                                   .append(industry_grouped_other)
industry_grouped_all

Sponsor,Department,Total
BASF Corporation,Chemistry Department,9069312
Aduro Biotech,Cancer Research Laboratory,7500000
BP Group,Energy Biosciences Institute,5000000
Intel Corporation,Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",The California Institute for Quantitative Biosciences,2904754
Chevron Corporation,Chemical and Biomolecular Engineering,2825291
Intel Corporation,Engineering Research Support Organization,2409614
AstraZeneca Inc.,Innovative Genomics Initiative,2340000
Wellspring Advisors LLC,Center of Evaluation for Global Action,1879998
GlaxoSmithKline,Chemical and Biomolecular Engineering,1810954


In [29]:
# Mapping the smaller donors in 'Others'
industry_other_grouped = industry_all.where('Sponsor', 'Other')\
                                     .groups(['Department', 'Sponsor Copy'], sum)\
                                     .drop(['Sponsor sum'])
    
industry_other_grouped

Department,Sponsor Copy,Total sum
Berkeley Seismological Laboratory,"Array Information Technology, Inc.",213855
Bioengineering,"Microbial Medical, Inc.",22500
Bioengineering,Novartis Institutes for BioMedical Research,377099
Bioengineering,OncoSec Medical Inc.,15650
Bioengineering,"Zymergen, Inc.",125600
Botanical Garden,Creekside Center for Earth Observation,11200
Botanical Garden,Vollmar Natural Lands Consulting,41000
CNR Office of the Dean,"Los Gatos Research, Inc.",67453
CNR Office of the Dean,Westat,10896
California Institute for Energy & Environment,Alstom,50000


In [30]:
# (cont.)
other_mapping = {}
targets = industry_other_grouped.column('Department')
sponsors = industry_other_grouped.column('Sponsor Copy')
total = industry_other_grouped.column('Total sum')
for i in range(industry_other_grouped.num_rows):
    sub = {}
    sub['sponsor'] = sponsors[i]
    sub['value'] = total[i]
    target = targets[i]
    if target in other_mapping.keys():
        other_mapping[target] += [sub]
    else:
        other_mapping[target] = [sub]
other_mapping  # JSON

{'Berkeley Seismological Laboratory': [{'sponsor': 'Array Information Technology, Inc.',
   'value': 213855}],
 'Bioengineering': [{'sponsor': 'Microbial Medical, Inc.', 'value': 22500},
  {'sponsor': 'Novartis Institutes for BioMedical Research', 'value': 377099},
  {'sponsor': 'OncoSec Medical Inc.', 'value': 15650},
  {'sponsor': 'Zymergen, Inc.', 'value': 125600}],
 'Botanical Garden': [{'sponsor': 'Creekside Center for Earth Observation',
   'value': 11200},
  {'sponsor': 'Vollmar Natural Lands Consulting', 'value': 41000}],
 'CNR Office of the Dean': [{'sponsor': 'Los Gatos Research, Inc.',
   'value': 67453},
  {'sponsor': 'Westat', 'value': 10896}],
 'California Institute for Energy & Environment': [{'sponsor': 'Alstom',
   'value': 50000},
  {'sponsor': 'Food Service Technology Center', 'value': 65000},
  {'sponsor': 'Pacific Gas and Electric Company', 'value': 450850}],
 'Center for Environmental Design Research': [{'sponsor': 'Autodesk, Inc.',
   'value': 18452},
  {'sponsor

In [31]:
mappings = json.loads(open("data/node_dictionary.json").read())
industry_json = nodes_links(industry_grouped_all, mappings) # JSON 

In [ ]:
mappings

In [37]:
def get_val(source_id):
    sum = 0
    for elem in industry_json['links']:
        if elem['source'] == source_id:
            sum += elem['value']
    return sum
    
agg_sponsortypes = {}
num = 0
for i in range(len(industry_json["nodes"])):
    elem = industry_json["nodes"][i]    
    if elem['name'] != "Other" and elem['type'] == "sponsors":
        num += 1
        val = get_val(i)
        if (not elem['cat']):
            print(elem)
        if elem['cat'] not in agg_sponsortypes.keys():
            agg_sponsortypes[elem['cat']] = val
        else:
            agg_sponsortypes[elem['cat']] += val

{'type': 'sponsors', 'name': 'New Profit, Inc.', 'abbrev': '', 'cat': ''}
{'type': 'sponsors', 'name': 'Oak Ridge National Laboratory', 'abbrev': '', 'cat': ''}


In [33]:
agg_sponsortypes

{'': 2402428,
 'Aerospace': 1640860,
 'Automotives': 7616473,
 'Biotechnology': 7926154,
 'Chemicals': 13454726,
 'Consultancy': 3165181,
 'Energy': 12987530,
 'Optometry': 660096,
 'Pharmaceuticals': 10822200,
 'Technology': 36316748,
 'Tobacco': 1725779}

In [71]:
industry.show()

Sponsor,Control Unit,Division,Department,Total
BASF Corporation,Colleges,College of Chemistry,Chemistry Department,9069312
Aduro Biotech,L&S College of Letters & Sciences,L&S Biological Sciences,Cancer Research Laboratory,7500000
"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,5000000
Intel Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),2904754
Chevron Corporation,Colleges,College of Chemistry,Chemical and Biomolecular Engineering,2825291
Intel Corporation,Colleges,College of Engineering,ERSO Engineering Research Support Organization,2409614
AstraZeneca Inc.,L&S College of Letters & Sciences,L&S Biological Sciences,Innovative Genomics Initiative,2340000
Wellspring Advisors LLC,Vice Chancellor - Research,VC Res Other Research Units,Center of Evaluation for Global Action,1879998
GlaxoSmithKline (Incl Smith Kline Beecham),Colleges,College of Chemistry,Chemical and Biomolecular Engineering,1810954


In [59]:
# Divisions
stem = ['CNR College of Natural Resources', 'Chemistry Department', 'College of Chemistry',
       'College of Engineering', 'College of Environmental Design', 'Division of Physical Sciences',
       'L&S Biological Sciences', 'School of Information', 'School of Optometry', 
       ]
other = ['Strategic Academic and Facilities Planning', 'UC Library', 
        'Vice Provost Academic Affairs & Faculty Welfare', 'Vice Chancellor - Student Affairs Division',
        'Vice Chancellor - Real Estate', 'Vice Chancellor - Administration', 
        'VP Academic Planning & Fac', 'Office of Equity & Inclusion',
         'EVCP Executive Vice Chancellor & Provost']

In [60]:
# if division or department includes sciences, chemistry, engineering, transportation, 
def non_stem(div, dept):
    keywords = ['social sciences', 'international studies', 'education', 'labor and employment',
               'law', 'haas', 'regional development', 'humanities']
    div_ = np.any([x in div.lower() for x in keywords])
    dept_ = np.any([x in dept.lower() for x in keywords])
    return div_ or dept_

In [72]:
stem_industry = industry.with_column('Non_Stem',
                                    industry.apply(non_stem, ['Division', 'Department']))
stem_industry.to_csv('industry_nonstem_stem.csv')

In [63]:
non = sum(stem_industry.where('Non_Stem', True).column('Total'))
stem = sum(stem_industry.where('Non_Stem', False).column('Total'))
print(non, "to non-stem from industry")
print(stem, "to stem from industry")
print(stem/non ,": 1 stem to non-stem ratio")

2360531 to non-stem from industry
122812493 to stem from industry
52.0274857649 : 1 stem to non-stem ratio
